## Data Loading

In [93]:
import pandas as pd
import io
df = pd.read_csv("/content/job_market.csv")
print(df.head())

                  job_title        company           location   job_type  \
0       Engineering Manager        DataInc  San Francisco, CA     Remote   
1       Engineering Manager  EnterpriseHub       New York, NY     Remote   
2       Engineering Manager     StartupXYZ        Seattle, WA  Part-time   
3             Lead Engineer    WebDynamics        Seattle, WA  Full-time   
4  Senior Software Engineer   DigitalWorks  San Francisco, CA  Full-time   

     category  salary_min  salary_max  experience_required publication_date  \
0  Technology      151082      291345                  4.0       2025-11-27   
1  Technology      156891      280075                  3.0       2025-11-27   
2  Technology      152134      280310                  4.0       2025-11-27   
3  Technology      151918      253988                  7.0       2025-11-27   
4  Technology      148141      252584                  9.0       2025-11-27   

                                              skills  
0  AWS, Agile

## Data Preprocessing

In [83]:
## 1. Find rows where company == "Bolt Technology" and change category into "Insurtech"
df.loc[df["company"] == "Bolt Technology", "category"] = "Insurtech"

## 2. Find publication_date != "2025-11-27" and change experience_required for Senior
## positions to 9.0, Junior positions to 3.0 and others to 4.0
## base mask
mask = df["publication_date"] != "2025-11-27"

## Senior: 9 years
df.loc[mask & (df["job_title"] == "Senior"), "experience_required"] = 9.0

## Junior: 3 years
df.loc[mask & (df["job_title"] == "Junior"), "experience_required"] = 3.0

## Others: 4 years
df.loc[mask & ~df["job_title"].isin(["Senior", "Junior"]), "experience_required"] = 4.0

## 3. Find job_type == berufserfahren to professional / experienced
df.loc[(df["job_type"] == "berufserfahren"), "job_type"] = "professional / experienced"

## 4. Find job_type == berufseinstieg to Intership
df.loc[(df["job_type"] == "berufseinstieg"), "job_type"] = "Internship"

## 5. Find job_type == NaN to Full-time
df.loc[(df["job_type"].isnull()), "job_type"] = "Full-time"

## 6. Find category == NaN to Technology
df.loc[(df["category"].isnull()), "category"] = "Technology"

## 7. Find publication_date != "2025-11-27" to "2025-11-27"
df.loc[df["publication_date"] != '2025-11-27', "publication_date"] = "2025-11-27"


## Data Exploration

Which company pays has the most job titles?

In [84]:
df.groupby("company")["job_title"].count().sort_values(ascending=False)

,job_title
company,
DataInc,25
EnterpriseHub,24
WebDynamics,23
TechCorp,23
CodeFactory,20
DigitalWorks,20
InnovateLab,17
CloudSystems,17
StartupXYZ,16


Which company pays the most per employee?

In [85]:
df.groupby("company")["salary_max"].mean().sort_values(ascending=False)

,salary_max
company,
Automat-it,190000.000000
Apaleo,190000.000000
1KOMMA5˚,180000.000000
Bearpaw GmbH,180000.000000
Skalar,180000.000000
SolarEdge,180000.000000
HHL Leipzig Graduate School of Management,180000.000000
Paymentology,180000.000000
fleeky GmbH,180000.000000


For DataInc, what are the job positions and lets see the position's maximum salary and years of experience required.

In [86]:
result_datainc = (
    df.loc[df["company"] == "DataInc"]
      .groupby("job_title", as_index=False)[["salary_max", "experience_required"]]
      .max()
).sort_values("salary_max", ascending=False)
print(result_datainc)

                    job_title  salary_max  experience_required
1         Engineering Manager      291345                  4.0
8             Project Manager      237154                  2.0
5               Lead Engineer      197589                  9.0
11          Software Engineer      193973                  5.0
7             Product Manager      193211                  3.0
2          Frontend Developer      192555                  2.0
0              Data Scientist      187248                  3.0
9                 QA Engineer      186282                  4.0
6   Machine Learning Engineer      186095                  3.0
10      Senior Data Scientist      184236                  9.0
12        Solutions Architect      173023                  4.0
14             Technical Lead      161933                  8.0
15                UX Designer      144503                  2.0
13       System Administrator      137346                  2.0
3        Full Stack Developer      130316              

For WebDynamic, what are the job positions and lets see the position's maximum salary and years of experience required.

In [87]:
result_webdynamic = (
    df.loc[df["company"] == "WebDynamics"]
      .groupby("job_title", as_index=False)[["salary_max", "experience_required"]]
      .max()
).sort_values("salary_max", ascending=False)
print(result_webdynamic)

                    job_title  salary_max  experience_required
4               Lead Engineer      253988                  7.0
10   Senior Software Engineer      248059                  8.0
9       Senior Data Scientist      244158                 11.0
14             Technical Lead      209806                 11.0
6             Product Manager      193038                  5.0
5   Machine Learning Engineer      190665                  5.0
7             Project Manager      182407                  4.0
2          Frontend Developer      178290                  4.0
8                 QA Engineer      154954                  4.0
12        Solutions Architect      151938                  3.0
3            Junior Developer      145045                  0.0
13       System Administrator      141984                  5.0
0           Backend Developer      139674                  3.0
11          Software Engineer      137099                  3.0
1            Business Analyst      106042              

For EnterpriseHub, what are the job positions and lets see the position's maximum salary and years of experience required.

In [88]:
result_enterprisehub = (
    df.loc[df["company"] == "EnterpriseHub"]
      .groupby("job_title", as_index=False)[["salary_max", "experience_required"]]
      .max()
).sort_values("salary_max", ascending=False)
print(result_enterprisehub)

                    job_title  salary_max  experience_required
3         Engineering Manager      280075                  3.0
7             Product Manager      233476                  5.0
11   Senior Software Engineer      199930                  9.0
10      Senior Data Scientist      197147                 11.0
13        Solutions Architect      189999                  4.0
9           Security Engineer      187098                  5.0
5               Lead Engineer      184145                 10.0
12          Software Engineer      181173                  4.0
4        Full Stack Developer      148405                  4.0
8                 QA Engineer      138893                  4.0
6   Machine Learning Engineer      132888                  4.0
2              Data Scientist      131764                  5.0
1                Data Analyst      107918                  5.0
14       System Administrator      101397                  3.0
0            Business Analyst       99605              

For CodeFactory, what are the job positions and lets see the position's maximum salary and years of experience required.

In [89]:
result_codefactory = (
    df.loc[df["company"] == "CodeFactory"]
      .groupby("job_title", as_index=False)[["salary_max", "experience_required"]]
      .max()
).sort_values("salary_max", ascending=False)
print(result_codefactory)

                   job_title  salary_max  experience_required
8   Senior Software Engineer      200220                 12.0
2             Data Scientist      196060                  2.0
5            Project Manager      194130                  4.0
0          Backend Developer      185900                  4.0
6                QA Engineer      174792                  3.0
4            Product Manager      171881                  4.0
11            Technical Lead      151875                  9.0
9        Solutions Architect      149586                  4.0
7          Security Engineer      136568                  3.0
3           Junior Developer      135378                  0.0
1           Business Analyst      124119                  4.0
10      System Administrator      107131                  5.0


Since I am more interested in the Data Scientist positions, lets compare the maz salary for each company for the job title and the years of experience along with the skills the company is demanding.

In [90]:
result_data = (
    df.loc[df["job_title"].isin(["Data Scientist", "Senior Data Scientist"])]
      .groupby(["company", "job_title", "experience_required"], as_index=False)
      .agg({
          "salary_max": "max",
          "skills": lambda x: sorted(set(x))  # unique skills; use list(x) if you want all
      })
).sort_values("salary_max", ascending=False)

print(result_data)

          company              job_title  experience_required  salary_max  \
12    WebDynamics  Senior Data Scientist                 11.0      244158   
2    CloudSystems  Senior Data Scientist                  6.0      200044   
1    CloudSystems  Senior Data Scientist                  5.0      197803   
8   EnterpriseHub  Senior Data Scientist                 11.0      197147   
3     CodeFactory         Data Scientist                  2.0      196060   
9     InnovateLab  Senior Data Scientist                  8.0      189378   
5         DataInc         Data Scientist                  3.0      187248   
6         DataInc  Senior Data Scientist                  9.0      184236   
10     StartupXYZ         Data Scientist                  2.0      152915   
0    CloudSystems         Data Scientist                  5.0      150171   
4         DataInc         Data Scientist                  2.0      148172   
11       TechCorp         Data Scientist                  4.0      139205   

Lets visualize this result!

In [91]:
import plotly.express as px

fig = px.bar(
    result_data,
    x="company",
    y="salary_max",
    title="Max Salary by Company"
)
fig.show()


What if I want to change my path? Lets check the highest paying positions in all the companies.

In [92]:
result_10 = (
    df.loc[df.groupby("job_title")["salary_max"].idxmax()]
      [["job_title", "salary_max", "company", "experience_required"]]
      .sort_values("salary_max", ascending=False)
      .head(10)
)
print(result_10)

                   job_title  salary_max        company  experience_required
0        Engineering Manager      291345        DataInc                  4.0
7              Lead Engineer      258933   AI Solutions                 11.0
4   Senior Software Engineer      252584   DigitalWorks                  9.0
6      Senior Data Scientist      244158    WebDynamics                 11.0
9            Project Manager      237154        DataInc                  2.0
10           Product Manager      233476  EnterpriseHub                  4.0
18            Technical Lead      209806    WebDynamics                 11.0
64        Frontend Developer      196212       TechCorp                  3.0
31            Data Scientist      196060    CodeFactory                  2.0
50         Backend Developer      194848    InnovateLab                  2.0


So the final call would be to not change my path and stick to Data Scientist positions since CodeFactory is compensating nearly 200K for 2 years of experience and then a jump to 240K with 11 years of experience at WebDynamics seems pretty fair, considering inflation and cost of living in Germany.